In [7]:
"""Import libraries"""
import random
from datetime import datetime
import csv

In [8]:
"""Base Sensor Class"""
class SafetySensor:
    def __init__(self, sensor_id, location, threshold, critical_threshold=None):
        self.sensor_id = sensor_id # Unique ID of the sensor
        self.location = location   # Where the sensor is placed (e.g. "Tunnel A")
        self.threshold = threshold # The danger threshold
        self.critical_threshold = critical_threshold  # The danger level
        self.current_value = 0  #Current sensor reading
    
    def update_reading(self, new_value):
        self.current_value = new_value
    
    def check_status(self):
        """Update the sensor's current reading"""
        if self.critical_threshold and self.current_value > self.critical_threshold:
            return "CRITICAL"
        elif self.current_value > self.threshold:
            return "WARNING"
        else:
            return "NORMAL"
    
    def get_alert_level(self):
        """Determine alert severity"""
        status = self.check_status()
        if status == "CRITICAL":
            return " CRITICAL"
        elif status == "WARNING":
            return " WARNING"
        else:
            return " NORMAL"
    
    def __str__(self):
        """Display sensor information"""
        return (f"{self.__class__.__name__} {self.sensor_id} at {self.location}: "
                f"{self.current_value} (Warning: {self.threshold}, "
                f"Critical: {self.critical_threshold}) | Status: {self.get_alert_level()}")


In [9]:
"""Specialized Sensors"""
class MethaneSensor(SafetySensor):
    def __init__(self, sensor_id, location):
        # Warning at 1%, Critical at 2%
        super().__init__(sensor_id, location, 1.0, 2.0)

class TemperatureSensor(SafetySensor):
    def __init__(self, sensor_id, location):
        # Warning at 40°C, Critical at 45°C
        super().__init__(sensor_id, location, 40, 45)

class AirQualitySensor(SafetySensor):
    def __init__(self, sensor_id, location):
        # Warning at 19% O2, Critical at 17%
        super().__init__(sensor_id, location, 19, 17)

In [10]:
"""Alert System"""
class AlertSystem:
    def __init__(self):
        self.alerts = []
        self.critical_count = 0
        self.warning_count = 0
    
    def add_alert(self, sensor):
        status = sensor.check_status()
        
        if status == "CRITICAL":
            alert_msg = f" CRITICAL ALERT! {sensor}"
            self.alerts.append(("CRITICAL", alert_msg))
            self.critical_count += 1
            print(alert_msg)
        elif status == "WARNING":
            alert_msg = f" WARNING: {sensor}"
            self.alerts.append(("WARNING", alert_msg))
            self.warning_count += 1
            print(alert_msg)
        else:
            print(f" NORMAL {sensor.sensor_id} at {sensor.location} Operating Normally")
    
    def display_alerts(self):
        if not self.alerts:
            print("\nAll Systems Normal - No Alerts")
        else:
            print(f"\n=== ALERT SUMMARY ===")
            print(f"CRITICAL Alerts: {self.critical_count}")
            print(f"WARNING Alerts: {self.warning_count}")
            
            print("\n=== DETAILED ALERTS ===")
            for i, (severity, alert) in enumerate(self.alerts, 1):
                print(f"{i}. {alert}")
    
    def clear_alerts(self):
        self.alerts = []
        self.critical_count = 0
        self.warning_count = 0

In [11]:
"""Initialize System"""
methane_sensor = MethaneSensor("CH4-001", "Tunnel A")
temp_sensor = TemperatureSensor("TEMP-001", "Tunnel B")
air_sensor = AirQualitySensor("AIR-001", "Tunnel C")

sensors = [methane_sensor, temp_sensor, air_sensor]
alert_system = AlertSystem()

In [12]:
"""Simulation Function"""

def run_simulation():
    print("\n" + "="*40)
    print("MINING SAFETY MONITORING SYSTEM")
    print("="*40)
    
    alert_system.clear_alerts()

    """Generate readings"""
    methane_sensor.update_reading(round(random.uniform(0.5, 2.5), 2))
    temp_sensor.update_reading(random.randint(35, 47))
    air_sensor.update_reading(random.randint(16, 21))

    """Check sensors and trigger appropriate responses"""
    for sensor in sensors:
        print(f"\n{sensor}")
        
        status = sensor.check_status()
        if status == "CRITICAL":
            print(" Initiating Emergency Protocols!")
        elif status == "WARNING":
            print(" Caution Required - Monitor Closely")
        else:
            print("✓ Within Safe Parameters")
        
        alert_system.add_alert(sensor)
    
    """System-wide status check"""
    if alert_system.critical_count > 0:
        print("\n EMERGENCY: Critical Alerts Detected! ")
    elif alert_system.warning_count > 0:
        print("\n Warning: Non-critical Alerts Present")
    else:
        print("\n All Systems Normal")
    
    alert_system.display_alerts()
    log_alerts()

In [13]:
"""Alert Logging with severity tracking"""

def log_alerts():
    filename = "mining_alerts.csv"
    with open(filename, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for severity, alert in alert_system.alerts:
            writer.writerow([
                datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                severity,
                alert.replace("🔴 CRITICAL ALERT! ", "").replace("🟠 WARNING: ", "")
            ])



In [14]:
"""Run Simulation"""
run_simulation()


MINING SAFETY MONITORING SYSTEM

MethaneSensor CH4-001 at Tunnel A: 0.56 (Warning: 1.0, Critical: 2.0) | Status:  NORMAL
✓ Within Safe Parameters
 NORMAL CH4-001 at Tunnel A Operating Normally

TemperatureSensor TEMP-001 at Tunnel B: 47 (Warning: 40, Critical: 45) | Status:  CRITICAL
 Initiating Emergency Protocols!
 CRITICAL ALERT! TemperatureSensor TEMP-001 at Tunnel B: 47 (Warning: 40, Critical: 45) | Status:  CRITICAL

AirQualitySensor AIR-001 at Tunnel C: 21 (Warning: 19, Critical: 17) | Status:  CRITICAL
 Initiating Emergency Protocols!
 CRITICAL ALERT! AirQualitySensor AIR-001 at Tunnel C: 21 (Warning: 19, Critical: 17) | Status:  CRITICAL

 EMERGENCY: Critical Alerts Detected! 

=== ALERT SUMMARY ===
CRITICAL Alerts: 2
WARNING Alerts: 0

=== DETAILED ALERTS ===
1.  CRITICAL ALERT! TemperatureSensor TEMP-001 at Tunnel B: 47 (Warning: 40, Critical: 45) | Status:  CRITICAL
2.  CRITICAL ALERT! AirQualitySensor AIR-001 at Tunnel C: 21 (Warning: 19, Critical: 17) | Status:  CRITICAL

In [15]:
"""Safety Checks"""
while True:
    run_simulation()
    choice = input("\nRun another simulation? (y/n): ").lower()
    
    if choice == 'y':
        continue
    elif choice == 'n':
        print("Shutting down safety monitoring system...")
        break
    else:
        print("Invalid input. Please enter 'y' or 'n'")


MINING SAFETY MONITORING SYSTEM

MethaneSensor CH4-001 at Tunnel A: 1.63 (Warning: 1.0, Critical: 2.0) | Status:  WARNING
 Caution Required - Monitor Closely

TemperatureSensor TEMP-001 at Tunnel B: 46 (Warning: 40, Critical: 45) | Status:  CRITICAL
 Initiating Emergency Protocols!
 CRITICAL ALERT! TemperatureSensor TEMP-001 at Tunnel B: 46 (Warning: 40, Critical: 45) | Status:  CRITICAL

AirQualitySensor AIR-001 at Tunnel C: 18 (Warning: 19, Critical: 17) | Status:  CRITICAL
 Initiating Emergency Protocols!
 CRITICAL ALERT! AirQualitySensor AIR-001 at Tunnel C: 18 (Warning: 19, Critical: 17) | Status:  CRITICAL

 EMERGENCY: Critical Alerts Detected! 

=== ALERT SUMMARY ===
CRITICAL Alerts: 2
WARNING Alerts: 1

=== DETAILED ALERTS ===
1.  WARNING: MethaneSensor CH4-001 at Tunnel A: 1.63 (Warning: 1.0, Critical: 2.0) | Status:  WARNING
2.  CRITICAL ALERT! TemperatureSensor TEMP-001 at Tunnel B: 46 (Warning: 40, Critical: 45) | Status:  CRITICAL
3.  CRITICAL ALERT! AirQualitySensor AIR-